In [21]:
## data generation
import sys, os
import matplotlib.pyplot as plt
sys.path.insert(0, os.getcwd())



In [19]:
pip install optax==0.1.7


  Attempting uninstall: optax
    Found existing installation: optax 0.1.8
    Uninstalling optax-0.1.8:
      Successfully uninstalled optax-0.1.8
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 3.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
!python3 -m venv .venv
!source .venv/bin/activate      # (Windows PowerShell: .\.venv\Scripts\Activate.ps1)
!pip install numpy


In [6]:
pip install -U jax jaxlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 1.8 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 MB 2.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.5 MB/s eta 0:00:00a 0:00:01
  Created wheel for jax: filename=jax-0.4.13-py3-none-any.whl size=1518704 sha256=dbbf3d3b2dbd69848959967f299880ebf4e58e8b674b4b5c360ba5b778de3c51
  Stored in directory: /Users/banshihan/Library/Caches/pip/wheels/46/d9/15/d2800d4089dc4c77299ac7513c6aa1036f5491edbd2bf6ba16
Successfully built jax
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install flax


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 2.9 MB/s eta 0:00:00a 0:00:01m
  Created wheel for msgpack: filename=msgpack-1.1.0-cp38-cp38-macosx_11_0_arm64.whl size=83182 sha256=9112e7a38080ae0a7a091a1b5f45fd30e3db31545714ee284be90eb489cdab0a
  Stored in directory: /Users/banshihan/Library/Caches/pip/wheels/55/aa/93/797450f0b3d3ac6906a2a32306efbb304940a5a8eb5bdff767
Successfully built msgpack
Note: you may need to restart the kernel to use updated packages.


## Linear Case

In [17]:
from datasets import LinearGaussianDataset, SigmoidDataset, SphereDataset
import numpy as np

# 设置随机种子
seed = 42

# 创建数据集实例
dataset = LinearGaussianDataset(seed=seed, dimension=2, padding_dimension=1)

# 生成数据批次
batch_size = 1000
data = dataset.get_batch(batch_size)  ##:contentReference[oaicite:22]{index=22}




## Linear case data preparation

In [ ]:
!pwd  # 看你当前在哪
!ls   # 确保能看到 seed_linpadding_expts.sh 和 run.py


/Users/banshihan/Documents/GitHub/homework-3-ShihanBan/Diffusion-model-low-dimensional-data
__init__.py              diffusion_utils.py       seed_linpadding_expts.sh
__pycache__              generated_data           training.py
data_generation.ipynb    model.py                 utils.py
datasets.py              networks.py
diffusion.py             run.py


In [26]:
# Notebook Cell 1: 导入
import numpy as np
from datasets import LinearGaussianDataset    # 原仓库中的类定义 :contentReference[oaicite:0]{index=0}



In [31]:
import numpy as np
from datasets import LinearGaussianDataset

# 固定 intrinsic dimension r* = 3
r_star = 3  

# 定义 ambient 维度到 padding_dim 的映射（对应脚本里的组合）
dd_to_paddings = {
    3: [9, 17],
    6: [6, 14],
    9: [3, 11],
    12:[8],      # 脚本里只对 12 维做了一个 padding_dim 组合
}

# 三个随机种子 ds ∈ {2,3,4}
seeds = [2, 3, 4]

# 样本数 N = num_batches * batch_size = 100000 * 128
N = 100_000 * 128  

# 存放所有生成结果的字典
data_store = {}

for dd, paddings in dd_to_paddings.items():
    for pad in paddings:
        for seed in seeds:
            key = f"d{dd}_pad{pad}_seed{seed}"
            # 实例化 Dataset
            ds = LinearGaussianDataset(
                seed=seed,
                dimension=dd,
                intrinsic_dimension=r_star,
                padding_dimension=pad,
                var_added=0.0
            )
            # 一次性生成 N 个样本
            X = ds.get_batch(N)  # jax.numpy array of shape (N, dd+pad)
            # 转为 NumPy 并存储
            data_store[key] = np.array(X)
            print(f"{key} → shape = {data_store[key].shape}")



d3_pad9_seed2 → shape = (12800000, 12)
d3_pad9_seed3 → shape = (12800000, 12)
d3_pad9_seed4 → shape = (12800000, 12)
d3_pad17_seed2 → shape = (12800000, 20)
d3_pad17_seed3 → shape = (12800000, 20)
d3_pad17_seed4 → shape = (12800000, 20)
d6_pad6_seed2 → shape = (12800000, 12)
d6_pad6_seed3 → shape = (12800000, 12)
d6_pad6_seed4 → shape = (12800000, 12)
d6_pad14_seed2 → shape = (12800000, 20)
d6_pad14_seed3 → shape = (12800000, 20)
d6_pad14_seed4 → shape = (12800000, 20)
d9_pad3_seed2 → shape = (12800000, 12)
d9_pad3_seed3 → shape = (12800000, 12)
d9_pad3_seed4 → shape = (12800000, 12)
d9_pad11_seed2 → shape = (12800000, 20)
d9_pad11_seed3 → shape = (12800000, 20)
d9_pad11_seed4 → shape = (12800000, 20)
d12_pad8_seed2 → shape = (12800000, 20)
d12_pad8_seed3 → shape = (12800000, 20)
d12_pad8_seed4 → shape = (12800000, 20)


In [32]:
# cell 1
import time
import numpy as np
import jax
import jax.numpy as jnp

# 确保项目模块可访问
import sys
sys.path.append('..')

from diffusion_model import UNet, GaussianDiffusion


In [33]:
# cell 2
def make_batches(X: np.ndarray, batch_size: int):
    """Yield successive batches from X (numpy array)."""
    N = X.shape[0]
    for i in range(0, N, batch_size):
        yield jnp.array(X[i : i + batch_size])


In [34]:
# cell 3
def train_on_dataset(
    X: np.ndarray,
    data_shape: tuple,
    seed: int,
    num_epochs: int = 50,
    batch_size: int = 128,
    timesteps: int = 1000,
    beta_start: float = 1e-4,
    beta_end: float = 0.02,
    lr: float = 1e-4,
):
    # 1) 初始化模型
    channels = [64, 128, 256]
    unet = UNet(channels=channels)
    diffusion = GaussianDiffusion(
        model=unet,
        timesteps=timesteps,
        beta_start=beta_start,
        beta_end=beta_end,
        lr=lr,
    )

    # 2) 训练循环
    key = jax.random.PRNGKey(seed)
    for epoch in range(1, num_epochs + 1):
        t0 = time.time()
        # 每个 epoch 重新打乱顺序（可选）
        perm = np.random.RandomState(seed + epoch).permutation(X.shape[0])
        X_shuffled = X[perm]

        for batch in make_batches(X_shuffled, batch_size):
            key, sk = jax.random.split(key)
            diffusion.state = diffusion.train_step(diffusion.state, batch, sk)

        # 简单打印进度
        print(f"Epoch {epoch:03d} finished in {time.time() - t0:.1f}s")

    return diffusion


In [38]:
# cell 4
results = {}
num_epochs = 50
batch_size = 128

for key, X in data_store.items():
    print(f"\n=== Training on dataset {key} ===")
    # X.shape = (N, D); data_shape = (D,)
    D = X.shape[1]
    diffusion_trained = train_on_dataset(
        X=X,
        data_shape=(D,),
        seed=int(key.split('_')[-1].replace('seed','')),  # 从 key 提取种子
        num_epochs=num_epochs,
        batch_size=batch_size,
        timesteps=1000,
        beta_start=1e-4,
        beta_end=0.02,
        lr=1e-4,
    )
    results[key] = diffusion_trained



=== Training on dataset d3_pad9_seed2 ===


ValueError: Number of groups (32) does not divide the number of channels (3).